# Pipeline ELT em Python

O objetivo deste projeto é fazer um pipeline básiso de ETL utilizando python importando dados de um formato csv, fazer a transformação criando uma nova coluna e depois exportar para um banco de dados.

## Importando Pacotes

In [1]:
import pandas as pd
import chardet
import sqlite3

## Extract

In [2]:
# Usando chardet para detectar a codificação
with open('dados/Football_Stadiums.csv', 'rb') as arquivo:
    encoding = chardet.detect(arquivo.read())

# A codificação é armazenada no atributo 'encoding' do resultado
encod = encoding['encoding']

In [3]:
# Importando dados
df = pd.read_csv('dados/Football_Stadiums.csv', encoding= encod)
df.head()

,S.No,Confederation,Stadium,City,HomeTeams,Capacity,Country,IOC
0,1,UEFA,Stadiumi Besëlidhja,Lezhë,Besëlidhja,7000,Albania,ALB
1,2,UEFA,Stadiumi Flamurtari,Vlorë,Flamurtari Vlorë,8200,Albania,ALB
2,3,UEFA,Stadiumi Laçi,Laçi,KF Laçi,5000,Albania,ALB
3,4,UEFA,Stadiumi Niko Dovana,Durrës,Teuta,12040,Albania,ALB
4,5,UEFA,Stadiumi Selman Stërmasi,Tirana,"KF Tirana, Dinamo, Partizani",9500,Albania,ALB


## Transform

In [4]:
# Criando nova coluna com as ordens da capacodade do estádio
df['Capacity_Order'] = df['Capacity'].rank(ascending= False).astype(int)
df.head()

,S.No,Confederation,Stadium,City,HomeTeams,Capacity,Country,IOC,Capacity_Order
0,1,UEFA,Stadiumi Besëlidhja,Lezhë,Besëlidhja,7000,Albania,ALB,1364
1,2,UEFA,Stadiumi Flamurtari,Vlorë,Flamurtari Vlorë,8200,Albania,ALB,1287
2,3,UEFA,Stadiumi Laçi,Laçi,KF Laçi,5000,Albania,ALB,1498
3,4,UEFA,Stadiumi Niko Dovana,Durrës,Teuta,12040,Albania,ALB,1062
4,5,UEFA,Stadiumi Selman Stërmasi,Tirana,"KF Tirana, Dinamo, Partizani",9500,Albania,ALB,1232


In [5]:
# Verificando se a ordem está correta
df = df.sort_values(['Capacity'], ascending= False)
df.head()

,S.No,Confederation,Stadium,City,HomeTeams,Capacity,Country,IOC,Capacity_Order
1744,1745,CONCACAF,Michigan Stadium,Ann Arbor,Wolverines,107601,United States of America,USA,1
1686,1687,CONCACAF,Beaver Stadium,University Park,Penn State,106572,United States of America,USA,2
1752,1753,CONCACAF,Ohio Stadium,Columbus,Buckeyes,104944,United States of America,USA,3
1724,1725,CONCACAF,Kyle Field,College Station,Aggies,102733,United States of America,USA,4
1745,1746,CONCACAF,Neyland Stadium,Knoxville,Volunteers,102455,United States of America,USA,5


In [6]:
# Filtrando somente pelos 100 maiores estádios
df = df.head(50)
df.shape

(50, 9)

## Load

In [7]:
# Criando conexão com o banco de dados
conn = sqlite3.connect('Top50Estadios.db')

In [8]:
# Criando um novo banco de dando
df.to_sql('dados/Top50Estadios.db', conn, if_exists= 'replace', index= False)

50

In [9]:
# Fechando conexão
conn.close()

In [10]:
# Salvando em um novo csv
df.to_csv('dados/Top50Estadios.csv', index= False)